# **Resolving Partially Ordered Traces Using Deep Learning** (LSTM)

|                        | BPI 2012| BPI 2014 | Traffic |
|------------------------|---------|----------|---------|
| \|A\|                  | 24      |  9       | 11      |
| #Traces                | 13087   | 41353    | 150370  |
| #Events                | 262200  | 369485   | 561470  |
| #Event Sets            | 248205  | 243186   | 549452  |
| #uncertain Seq's       | 14      | 24       | 25      |
| Trace Uncertainty      | 38%     | 93%      |  6%     |
| Event Uncertainty      |  5%     | 40%      |  2%     |
| max(len(unc.seq))      |  4      |  4       |  3      |
| avg(len(unc.seq))      |  2.4    |  2.6     |  2.0    |

### imports and PIP installs

In [ ]:
!pip install pm4py

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/MyDrive/Bachelor_Thesis/')

In [4]:
import utils
from encoder import Encoder1, Encoder2, Encoder3

In [5]:
from pprint import pprint
from tqdm import tqdm
from random import randint, shuffle
from pm4py.objects.log.importer.xes import importer as xes_importer

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Dense, Dropout, Input, LSTM, TimeDistributed

In [6]:
from itertools import product

### Loading the logs

In [21]:
# real life logs

#b12_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/BPI_Challenge_2012.xes")
b14_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/BPI_Challenge_2014.xes")
traffic_log = xes_importer.apply("/content/drive/MyDrive/Bachelor_Thesis/logs/traffic_fines.xes")

In [48]:
# artificial logs

#a_log0   = xes_importer.apply("./logs/generated_logs/1561989897361-4_0.xes")
#a_log25  = xes_importer.apply("./logs/generated_logs/1561989897361-4_25.xes")
#a_log50  = xes_importer.apply("./logs/generated_logs/1561989897361-4_50.xes")
a_log75  = xes_importer.apply('/content/drive/MyDrive/Bachelor_Thesis/logs/generated_logs/1561989897313-3_75.xes')
#a_log100 = xes_importer.apply("./logs/generated_logs/1561989906794-495_100.xes")

In [40]:
# artifical log with test set >= 10k

a_log75  = xes_importer.apply('/content/drive/MyDrive/Bachelor_Thesis/logs/generated_logs_10k_uncertainty/1561989897100-0_75.xes')

In [11]:
def avg_trace_length(log):
  total = 0
  for trace in log:
    total += len(trace)

  return total / len(log)

def trace_frequencies(log):
  trace_freq = {}
  for trace in log:
    activity_seq = tuple([event['concept:name'] for event in trace])
    trace_freq[activity_seq] = trace_freq.get(activity_seq, 0) + 1

  return trace_freq


from math import factorial
def num_unc_eve(len_A, K):
  n = 0

  for k in range(1,K+1):
    numerator   =   factorial(len_A + k - 1) 
    denominator =   factorial(k) * factorial(len_A - 1)
    n += numerator / denominator

  return n

def num_pos_res(len_A, K):
  n = 0
  for k in range(1,K+1):
    n += len_A**k

  return n

In [52]:
len_A = 17
K = 4
num_unc_eve(len_A, K), num_pos_res(len_A, K)

(5984.0, 88740)

In [53]:
event_sets = {}
for trace in u_log_set:
  for e in trace:
    e_set = tuple(sorted(e))
    event_sets[e_set] = event_sets.get(e_set, 0) + 1

In [54]:
event_sets, len(event_sets)

({('a10',): 196,
  ('a10', 'a10'): 6,
  ('a10', 'a10', 'tau join9'): 1,
  ('a10', 'c13'): 2,
  ('a10', 'd16'): 3,
  ('a10', 'e14'): 6,
  ('a10', 'e14', 'tau join9'): 1,
  ('a10', 'f15'): 4,
  ('a10', 'g12'): 1,
  ('a10', 'h11'): 17,
  ('a10', 'h11', 'tau split8'): 3,
  ('a10', 'tau join9'): 10,
  ('a10', 'tau split8'): 21,
  ('a2',): 163,
  ('a2', 'a2'): 6,
  ('a2', 'b6'): 5,
  ('a2', 'b6', 'd7'): 1,
  ('a2', 'c5'): 1,
  ('a2', 'c5', 'h3'): 1,
  ('a2', 'd7'): 7,
  ('a2', 'g4'): 4,
  ('a2', 'h3'): 24,
  ('a2', 'h3', 'tau join1'): 1,
  ('a2', 'h3', 'tau join1', 'tau split0'): 1,
  ('a2', 'h3', 'tau split0'): 2,
  ('a2', 'tau join1'): 7,
  ('a2', 'tau split0'): 18,
  ('b6',): 145,
  ('b6', 'b6'): 1,
  ('b6', 'c5'): 21,
  ('b6', 'c5', 'd7'): 1,
  ('b6', 'c5', 'g4'): 2,
  ('b6', 'c5', 'tau split0'): 1,
  ('b6', 'd7'): 27,
  ('b6', 'd7', 'h3'): 1,
  ('b6', 'd7', 'tau join1'): 1,
  ('b6', 'g4'): 2,
  ('b6', 'h3'): 5,
  ('b6', 'h3', 'tau split0'): 1,
  ('b6', 'tau join1'): 1,
  ('b6', 'tau spl

In [47]:
100/5984 * 119

1.9886363636363638

# **Vanilla LSTM**
--- 

In [49]:
log = utils.remove_timezones(a_log75)

In [50]:
utils.abstract_time(log, utils.abstract_seconds) # AF
sparse_log = utils.get_sparse_log(log)
A = list(set([event["concept:name"] for trace in log for event in trace]))
c_log, u_log = utils.split_log(log)

A_set = [[activity] for activity in A]

#log_set = utils.get_sparse_log_set(log)
#c_log_set = utils.get_sparse_log_set(c_log)
#u_log_set = utils.get_sparse_log_set(u_log)

log_set = utils.get_sparse_log_set_artificial(log)     # AF
c_log_set = utils.get_sparse_log_set_artificial(c_log)
u_log_set = utils.get_sparse_log_set_artificial(u_log)

enc1 = False

In [51]:
max_trace_len = utils.longest_trace(log)
max_unc_trace_len = utils.longest_trace(log_set)
max_seq_len = utils.longest_unc_seq(log_set)
k = max_seq_len # longest uncertain sequences
print("Longest uncertain sequence of length:", k)

Longest uncertain sequence of length: 4


In [ ]:
sum = 0
count = 0
unc_seqs = []
for trace in u_log_set:
  for event_set in trace:
    unc_seqs.append(tuple(event_set))
    if len(event_set) > 1:
      sum += len(event_set)
      count += 1

unc_seqs = list(set(unc_seqs))
unc_seqs, len(unc_seqs)

In [ ]:
n_events = 0
for trace in log:
  for event in trace: 
    n_events += 1

n_events

8502

In [ ]:
NAME = "concept:name"
TIME = "time:timestamp"
for event in log[555]: 
  print("{:20} at   {}".format(event[NAME], event[TIME]))

Open                 at   2014-01-22 21:59:55
Assignment           at   2014-01-22 22:02:34
Status Change        at   2014-01-22 22:02:34
Closed               at   2014-01-22 22:11:01


In [ ]:
unc_seq = utils.possible_uncertain_seq(A, k) 
pos_res = utils.possible_resolutions(A, k)

In [ ]:
pos_res_for_unc_seq = utils.pos_res_for_unc_seq(unc_seq)

In [ ]:
max_trace_len, max_unc_trace_len, max_seq_len

(167, 91, 4)

### Feature Space Reduction

In [ ]:
ca, ua = utils.check_unc_activities(log_set, A) # the set of activities that do not appear in unc event set (ca) and the ones that do (ua)

In [ ]:
ca, ua

([],
 ['Open',
  'Assignment',
  'Status Change',
  'Closed',
  'Caused By CI',
  'Operator Update',
  'Update',
  'Quality Indicator Fixed',
  'Reassignment'])

In [ ]:
# rebuild unc_seq & pos_res without the events in ca set (the set that contains only activities that do not appear in unc event sets)
new_unc_seq = utils.possible_uncertain_seq(ua, k)
new_pos_res = utils.possible_resolutions(ua, k)

# add the certain activities to the set of uncertain sequences / possivle resolutions
cas = [[certain_act] for certain_act in ca]

new_unc_seq = new_unc_seq + cas
new_pos_res = new_pos_res + cas

In [ ]:
unc_seq = new_unc_seq
pos_res = new_pos_res

### Abstracting Timestamp Inforamtion


### Encoding 1

In [ ]:
sparse_u_log = utils.get_sparse_log(u_log)

In [ ]:
# shuffling before encoding, due to ambiguous decoding
c = list(zip(u_log_set, sparse_u_log))
shuffle(c)
u_log_set, sparse_u_log = zip(*c)
u_log_set, sparse_u_log = list(u_log_set), list(sparse_u_log)

enc1 = True

if len(u_log_set) > 10000:
    u_log_set = u_log_set[:10000]
    sparse_u_log = sparse_u_log[:10000]

In [ ]:
INenc = Encoder1(A, max_trace_len, True, False)
OUTenc = Encoder1(A, max_trace_len, False, True)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(sparse_u_log)

In [ ]:
enc_inputs.shape, enc_targets.shape

((10000, 167, 9), (10000, 167, 9))

### Encoding 2

In [ ]:
if len(u_log_set) > 10000:
    shuffle(u_log_set)
    u_log_set = u_log_set[:10000]

In [ ]:
INenc = Encoder2(unc_seq, max_trace_len, True)
OUTenc = Encoder2(pos_res, max_trace_len, False)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(u_log_set)

### Encoding 3

In [ ]:
if len(u_log_set) > 10000:
    shuffle(u_log_set)
    u_log_set = u_log_set[:10000]

In [ ]:
INenc = Encoder3(unc_seq, max_trace_len, True)
OUTenc = Encoder3(pos_res, max_trace_len, False)

enc_inputs = INenc.one_hot_encode_log(u_log_set)
enc_targets = OUTenc.one_hot_encode_log(u_log_set)

## Model (Preprocessing, Training, Predictions, Evaluation)

In [ ]:
# prepare specification values
n_samples = enc_inputs.shape[0]    # len(log_set)
mtl = enc_inputs.shape[1]
n_features_in = enc_inputs.shape[2]    # len(unc_seq)
n_features_out = enc_targets.shape[2]    # len(pos_res)

X, y = enc_inputs, enc_targets

print(n_samples, mtl, n_features_in, n_features_out)
print(enc_inputs.shape, enc_targets.shape)

10000 167 9 9
(10000, 167, 9) (10000, 167, 9)


In [ ]:
# if encoding1, then set shuffle=False since the encoding 1 has some ambiguouities
# --> we need the indexes of the unencoded data to check if a prediction corresponds to 
# the ground truth (i.e. check if the predicted, decoded vector matches the unencoded vector)

if enc1:
    cut = int(n_samples*0.8)
    X_train_full, X_test, y_train_full, y_test = X[:cut], X[cut:], y[:cut], y[cut:] 

else:
    X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 


# split train into train and validation data
valid_index = int(len(X_train_full)/100*10)          # make valid data 10% and train data the rest
X_valid, X_train = X_train_full[:valid_index], X_train_full[valid_index:]
y_valid, y_train = y_train_full[:valid_index], y_train_full[valid_index:]

# set up model
n_neurons = mtl
n_epoch = 30
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model = Sequential()
model.add(LSTM(n_neurons, input_shape=(mtl, n_features_in), return_sequences=True))
model.add(TimeDistributed(Dense(n_features_out, activation='softmax')))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])# maybe use loss='categorical_crossentropy'
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 167, 167)          118236    
_________________________________________________________________
time_distributed_8 (TimeDist (None, 167, 9)            1512      
Total params: 119,748
Trainable params: 119,748
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
history = model.fit(X_train, y_train, epochs=n_epoch, validation_data=(X_valid, y_valid), callbacks=[callback])

Epoch 1/30
225/225 [==============================] - 63s 274ms/step - loss: 0.0672 - categorical_accuracy: 0.5293 - val_loss: 0.0354 - val_categorical_accuracy: 0.0375
Epoch 2/30
225/225 [==============================] - 60s 268ms/step - loss: 0.0315 - categorical_accuracy: 0.1949 - val_loss: 0.0306 - val_categorical_accuracy: 0.0380
Epoch 3/30
225/225 [==============================] - 66s 295ms/step - loss: 0.0295 - categorical_accuracy: 0.0869 - val_loss: 0.0297 - val_categorical_accuracy: 0.0379
Epoch 4/30
225/225 [==============================] - 63s 279ms/step - loss: 0.0291 - categorical_accuracy: 0.0624 - val_loss: 0.0295 - val_categorical_accuracy: 0.0381
Epoch 5/30
225/225 [==============================] - 61s 273ms/step - loss: 0.0288 - categorical_accuracy: 0.0381 - val_loss: 0.0294 - val_categorical_accuracy: 0.0381
Epoch 6/30
225/225 [==============================] - 62s 276ms/step - loss: 0.0287 - categorical_accuracy: 0.0381 - val_loss: 0.0291 - val_categorical_acc

In [ ]:
model.save('/content/drive/MyDrive/Bachelor_Thesis/trained_models/LSTM_enc1_bpi14.h5')

In [ ]:
# decode , Xtest, ytest
if enc1:
  # get new test samples 
  # pass # decoding was already done
  dec_X_test = u_log_set[cut:]
  dec_y_test = sparse_u_log[cut:]
else:
    dec_X_test = utils.decode_X(X_test, INenc.idx_to_activity, mode='enc3') # mode: event or event_set
    dec_y_test = utils.decode_y(y_test, OUTenc.idx_to_activity, mode='enc2+3') # mode: event or event_set

In [ ]:
A

In [ ]:
import statistics
for act in A:
  print(round(statistics.mean(prediction_probabilities[act]), 2))

In [ ]:
for act in A:
  print(round(statistics.mean(actual_resolution_probabilities[act]), 2))

In [ ]:
prediction_probabilities

In [ ]:
actual_resolution_probabilities

In [ ]:
import pickle

a_file = open("LSTM1_ENC1_ART1_pred_prob.pkl", "wb")
pickle.dump(prediction_probabilities, a_file)
a_file.close()

a_file = open("LSTM1_ENC1_ART1_target_prob.pkl", "wb")
pickle.dump(actual_resolution_probabilities, a_file)
a_file.close()

#a_file = open("LSTM1_ENC1_ART1.pkl", "rb")
#output = pickle.load(a_file)
#a_file.close()

In [ ]:
prediction_probabilities = {}  # contains the probabilities for certain predictions of 
actual_resolution_probabilities = {}
eval_test_1(model, X_test, y_test, dec_X_test, dec_y_test, OUTenc.idx_to_activity, pos_res_for_unc_seq)

  0%|          | 3/2000 [00:00<13:49,  2.41it/s]

True unc set: ['Assignment', 'Status Change'] 0.25331069464866474
Predicted unc set: ['Assignment', 'Status Change'] 0.25331069464866474
True unc set: ['Closed', 'Caused By CI'] 0.23513674020534836
Predicted unc set: ['Caused By CI', 'Closed'] 0.2652147707657093
Source          : [['Open'], ['Assignment', 'Status Change'], ['Closed', 'Caused By CI']]
Predictedt Trace: ['Open', 'Assignment', 'Status Change', 'Caused By CI', 'Closed']
Target Trace    : ['Open', 'Assignment', 'Status Change', 'Closed', 'Caused By CI']

True unc set: ['Operator Update', 'Assignment', 'Status Change'] 0.022102180272912036
Predicted unc set: ['Operator Update', 'Status Change', 'Assignment'] 0.059373768650334675
Source          : [['Open'], ['Operator Update', 'Assignment', 'Status Change'], ['Closed']]
Predictedt Trace: ['Open', 'Operator Update', 'Status Change', 'Assignment', 'Closed']
Target Trace    : ['Open', 'Operator Update', 'Assignment', 'Status Change', 'Closed']

True unc set: ['Assignment', 'Upd

  0%|          | 7/2000 [00:00<07:55,  4.19it/s]

True unc set: ['Assignment', 'Status Change'] 0.25331069464866474
Predicted unc set: ['Assignment', 'Status Change'] 0.25331069464866474
True unc set: ['Caused By CI', 'Quality Indicator Fixed'] 0.9907241587713003
Predicted unc set: ['Caused By CI', 'Quality Indicator Fixed'] 0.9907241587713003
Source          : [['Open'], ['Assignment', 'Status Change'], ['Caused By CI', 'Quality Indicator Fixed']]
Predictedt Trace: ['Open', 'Assignment', 'Status Change', 'Caused By CI', 'Quality Indicator Fixed']
Target Trace    : ['Open', 'Assignment', 'Status Change', 'Caused By CI', 'Quality Indicator Fixed']

True unc set: ['Reassignment', 'Assignment', 'Operator Update'] 0.05018792779827979
Predicted unc set: ['Reassignment', 'Operator Update', 'Assignment'] 0.06031929001257966
True unc set: ['Assignment', 'Operator Update'] 0.222524879217481
Predicted unc set: ['Operator Update', 'Assignment'] 0.26959773822795796
Source          : [['Open'], ['Assignment'], ['Reassignment', 'Assignment', 'Opera

  0%|          | 9/2000 [00:01<06:15,  5.30it/s]

...

100%|█████████▉| 1999/2000 [02:33<00:00, 12.57it/s]

True unc set: ['Update', 'Assignment', 'Reassignment'] 0.16935756599529211
Predicted unc set: ['Update', 'Assignment', 'Reassignment'] 0.16935756599529211
True unc set: ['Caused By CI', 'Closed'] 0.9167094622277077
Predicted unc set: ['Caused By CI', 'Closed'] 0.9167094622277077
Source          : [['Open'], ['Status Change'], ['Operator Update'], ['Status Change'], ['Update'], ['Update', 'Assignment', 'Reassignment'], ['Operator Update'], ['Closed'], ['Caused By CI', 'Closed']]
Predictedt Trace: ['Open', 'Status Change', 'Operator Update', 'Status Change', 'Update', 'Update', 'Assignment', 'Reassignment', 'Operator Update', 'Closed', 'Caused By CI', 'Closed']
Target Trace    : ['Open', 'Status Change', 'Operator Update', 'Status Change', 'Update', 'Update', 'Assignment', 'Reassignment', 'Operator Update', 'Closed', 'Caused By CI', 'Closed']

True unc set: ['Status Change', 'Assignment'] 0.24397222348429892
Predicted unc set: ['Assignment', 'Status Change'] 0.25331069464866474
Source   

100%|██████████| 2000/2000 [02:33<00:00, 12.99it/s]

True unc set: ['Status Change', 'Assignment'] 0.24397222348429892
Predicted unc set: ['Assignment', 'Status Change'] 0.25331069464866474
Source          : [['Open'], ['Status Change', 'Assignment'], ['Closed']]
Predictedt Trace: ['Open', 'Assignment', 'Status Change', 'Closed']
Target Trace    : ['Open', 'Status Change', 'Assignment', 'Closed']



(0.302, 0.0)

In [ ]:
def eval_test_1(model, X_test, y_test, dec_X_test, dec_y_test, idx_to_act: dict, pos_res_for_unc_seq: dict) -> float:
  n_samples = X_test.shape[0]
  count = 0
  count_highest_prob_is_non_pos_res = 0
  n_event_sets = sum([1 for trace in dec_X_test for event_set in trace])
  
  for i in tqdm(range(n_samples)): #go over every trace in the evaluation log
    #get a prediction for the current trace
    y_hat = model.predict(X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1]))[0]
    dec_y_hat = [idx_to_act[np.argmax(y_hat[j])] for j in range(len(dec_y_test[i]))]

    is_equal, amount_of_repredicts = eval_trace_1(y_hat, dec_y_hat, y_test[i], dec_X_test[i], dec_y_test[i])
    count_highest_prob_is_non_pos_res += amount_of_repredicts
    
    if is_equal:
      count += 1
  
  return count / n_samples, count_highest_prob_is_non_pos_res / n_event_sets

def eval_trace_1(enc_y_hat, dec_y_hat, enc_y_test, dec_X_test, dec_y_test) -> bool:
  """ Returns true if the predicted possible resolution with the highest associated probability equals the true resolution."""
  predicted_trace = []
  n_event_sets = len(dec_X_test)                                              # number of event sets in the input trace
  n_events = enc_y_hat.shape[1]                                               # number of events in the prediction
  n_repredicts = 0

  e_set_len = [len(event_set) for event_set in dec_X_test]                   # length of each uncertain set
  indices = [0 for length in e_set_len]                                      # start indices of the uncertain set in the y_trace e.g. [[A,B], [C], [D,E]]
                                                                             #                                                   with [A, B, C, D, E] -> [0, 2, 3]
  for i in range(len(e_set_len[:-1])):
    indices[i+1] = indices[i] + e_set_len[i]

  for i in range(len(indices)): # it over all event sets
    
    if e_set_len[i] == 1: # append the identity mapping

      # check if the identity mapping was done correctly
      prediction = dec_y_hat[indices[i]]
      index = OUTenc.activity_to_idx[prediction]
      if prediction != dec_y_test[indices[i]]:
        prediction = dec_y_test[indices[i]]
        index = OUTenc.activity_to_idx[prediction]
        n_repredicts += 1

      predicted_trace.append(prediction)
      prob1 = enc_y_hat[indices[i]][index]
      prediction_probabilities[prediction] = prediction_probabilities.get(prediction, []) + [prob1]

      #print("True identity     :", dec_y_test[indices[i]])
      #print("Predicted identity:", dec_y_hat[indices[i]])

      # get the probability associated to the groudn truth
      truth = dec_y_test[indices[i]]
      index_of_truth = OUTenc.activity_to_idx[truth]
      prob2 = enc_y_hat[indices[i]][index_of_truth]
      actual_resolution_probabilities[truth] = actual_resolution_probabilities.get(truth, []) + [prob2]
      #print(index, index_of_truth)
      #print(prob1, prob2)
    
    else: # the e_set_len is >= 2
      
      possible_resolutions = pos_res_for_unc_seq[tuple(sorted(dec_X_test[i]))] # calc all possible resolutions for that event set
      predict_probs = {}
      pos_res_found = False 

      #compute the porbability for the ground truth accoridng to the model
      prob = 1
      true_set = []
      for j in range(e_set_len[i]):
        # check what is here / indices[i]+j in dec_y_test for an activity
        truth = dec_y_test[indices[i]+j]
        # get the corresponding index
        index_of_truth = OUTenc.activity_to_idx[truth]
        # get the corresponding prob and multiply it
        prob *= enc_y_hat[indices[i]+j][index_of_truth]
        # add true act str to set
        true_set.append(truth)
      #print("True unc set:", true_set, prob)
      actual_resolution_probabilities[tuple(true_set)] = actual_resolution_probabilities.get(tuple(true_set), []) + [prob]
      
      while not pos_res_found:
        
        idxs = [] # these are the current argmax idx to set their prob to zero for the next iteration

        for j in range(e_set_len[i]):
          # obtain the (at the moment) highest prob predicted event for the j-th element of the event set
          idx = np.argmax(enc_y_hat[indices[i]+j])
          idxs.append(idx)
          pred_event = OUTenc.idx_to_activity[idx]
          pred_prob = enc_y_hat[indices[i]+j][idx]   
          
          # store as (prediction, prob)
          predict_probs[j] = predict_probs.get(j, []) + [(pred_event,pred_prob)] 
          

        # compute all sequences + prob that can be built from the current predictions
        abs_seqs = list(dict_product(predict_probs))
        pred_seqs = build_sequences(abs_seqs)     
        
        # sort by prob
        pred_seqs.sort(key=lambda x: x[1], reverse=True)        # of the form [(['A', 'B'], 0.3), (), ...]
        
        if len(pred_seqs) == 2: # if a second sequence needed to be predicted, i.e. the first was not an actual pos res
          n_repredicts += 1

        # check if on of these is actually a pos res
        for pred_seq in pred_seqs:
          if pred_seq[0] in possible_resolutions:
            pos_res_found = True
            #print("Predicted unc set:", pred_seq[0], pred_seq[1])
            prediction_probabilities[tuple(pred_seq[0])] = prediction_probabilities.get(tuple(pred_seq[0]), []) + [pred_seq[1]]  # also append the prediction to the total predictions
            for event in pred_seq[0]:
              predicted_trace.append(event)
            break

        # if a possible resolution was found stop searching
        # else set the last max indices to have probability 0, to obain the next highest probabilities
        if pos_res_found:
          break
        else: 
          for j in range(len(idxs)):
            enc_y_hat[indices[i]+j][idxs[j]] = 0.0

  print('Source          :', dec_X_test)
  print('Predictedt Trace:', predicted_trace)
  print('Target Trace    :', dec_y_test)
  print()
  return predicted_trace == dec_y_test, n_repredicts

def dict_product(inp) -> list():
  """For a given dict returns the cartesian product of the keys as a list of dicts."""
  return (dict(zip(inp.keys(), values)) for values in product(*inp.values()))
  
def build_sequences(abs_seqs):
  """ Takes the structured created by 'dict_product' and returns a list of tuples containing the sequence and its probability."""
  pos_seqs = []
  for abs_seq in abs_seqs:
    seq = []
    prob = 1.0
    for key in abs_seq:
      seq.append(abs_seq[key][0])
      prob *= abs_seq[key][1]
    pos_seqs.append((seq, prob))

  return pos_seqs


In [ ]:
def plain_eval(model, X_test, y_test) -> float:
  n_samples = X_test.shape[0]
  count = 0

  for i in tqdm(range(n_samples)): # go over every trace in the evaluation log
    y_hat = model.predict(X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1]))

    if np.all(y_hat == y_test[i]):
      count += 1

  return count / n_samples

In [ ]:
plain_eval(model, X_test, y_test)

In [ ]:
def eval_test_enc1(model, X_test, dec_y_test, dec_X_test, idx_to_act: dict, pos_res_for_unc_seq: dict) -> float:
    total = X_test.shape[0]
    count = 0
    for i in tqdm(range(total)): #go over every trace in the evaluation log
        
        #get a prediction for the correct ordering of the current trace
        result = model.predict(X_test[i].reshape(1, X_test[i].shape[0], X_test[i].shape[1]))
        predicted_trace = []
        
        n_event_sets = len(dec_X_test[i]) # number of event sets in the input trace
        n_events = result.shape[1]        # number of events     in the prediction
        old_len = 0 # length of the previous event set, for corrext indexing
        
        if i == 13:
            print(count/i)
        
        for j in range(n_event_sets):
            len_event_set = len(dec_X_test[i][j])
            pos_res_predicted = False
            if np.all(X_test[i][j] == 0.0): # ignore padding predictions
                predicted_trace.append('-')
            else:
                idxs = []
                pred_eve_by_prob_order = {}
                for k in range(len_event_set):
                    idx = np.argmax(result[0][j+k+old_len]) # get idx prediction with highest prob
                    idxs.append(idx)
                    predicted_event = OUTenc.idx_to_activity[idx] # decode into event / sequence 
                    pred_eve_by_prob_order[k] = [predicted_event]
                    

                as_list = [a for a in pred_eve_by_prob_order.values()] # get the dict entries as lists
                predicted_sequences = [list(pred) for pred in product(*as_list)]
                
                if i == 13:
                    print("Predicted:", predicted_sequences)
                    print("Pos-res:", pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][j]))])
                    print(pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][j]))])
                    print()
                
                for seq in predicted_sequences:
                    if seq in pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][j]))]:
                      pos_res_predicted = True
                      predicted_events_for_event_set = seq
                      break
                
                # check if the prediction is actually a resolution
                while not pos_res_predicted:
                    # if not take the prediction with the 2nd highest prob... etc.
                    for k in range(len_event_set):
                        result[0][j+k+old_len][idxs[k]] = 0.0 # set the old idx with max prob to zero
                    
                    idxs = []
                    for k in range(len_event_set):
                        idx = np.argmax(result[0][j+k+old_len]) # get idx prediction with highest prob
                        idxs.append(idx)
                        predicted_event = OUTenc.idx_to_activity[idx] # decode into event / sequences
                        pred_eve_by_prob_order[k] += [predicted_event]
                    
                    
                    as_list = [a for a in pred_eve_by_prob_order.values()] # get the dict entries as lists
                    predicted_sequences = [list(pred) for pred in product(*as_list)]
                    
                    if i == 13:
                        print("Predicted:", predicted_sequences)
                        print("Pos-res:", pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][j]))])
                        print()
                    
                    for seq in predicted_sequences:
                        if seq in pos_res_for_unc_seq[tuple(sorted(dec_X_test[i][j]))]:
                            pos_res_predicted = True
                            predicted_events_for_event_set = seq
                            break
                    

                predicted_trace += predicted_events_for_event_set
                old_len = len_event_set

        predicted_trace = predicted_trace[:len(dec_y_test[i])]
        
        #print()
        #print()
        #print("Prediction: ", predicted_trace)
        #print("Truth: ", dec_y_test[i])
        #print("-----------------------------------------")
        if predicted_trace == dec_y_test[i]:
            count += 1

    return count / total